In [1]:
import random
import math as math
import sympy as sympy
import numpy as np
from seccap_constants190624 import *
#参照関数
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

def power(base, exponent, n):
    base = sympy.Integer(base)
    exponent = sympy.Integer(exponent)
    is_neg = exponent < 0
    if is_neg:
        exponent = -exponent
    n = sympy.Integer(n)
    x = sympy.Integer(1)
    e = exponent
    c = sympy.Mod(base, n)
    vals = [c]
    x += 1
    while x <= exponent:
        c = sympy.Mod(c**2, n)
        vals.append(c)
        x *= 2

    x /= 2
    answer = sympy.Integer(1)
    while len(vals) > 0:
        nextv = vals.pop()
        if x <= e:
            answer = sympy.Mod(nextv*answer, n)
            e -= x
            if e == 0:
                break
        x /= 2
    if is_neg:
        answer = modinv(answer, n)
    return answer

def multiplicative_inverse(a, n):
    g, x, y = egcd(a, n)
    if g != 1:
        raise Exception('逆元は存在しない')
    else:
        return x % n
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True

#ここから本プログラムの関数
def generate_keypair(p, q):
    n =p*q
    phi = (p-1) * (q-1)

    #phi(n)と互いに素のeをランダムに選択する。
    e = random.randrange(1, phi)

    #Euclid's Algorithmよりe、phi(n)は互いに素かを確かめる
    g = gcd(e, phi)
    while g != 1:
        e = random.randrange(1, phi)
        g = gcd(e, phi)

    #拡張Euclid's Algorithmより秘密鍵を生成（rsaの指定通り）
    d = multiplicative_inverse(e, phi)
    
    #公開鍵を(e, n)の形で、秘密鍵を(d, n)の形で戻す
    return ((e, n), (d, n))

def encrypt(pk, m):
    #2次元ベクトルpkから公開鍵e、nの値を得る
    e, n = pk
    #テキストm本文を式 m^e mod nによって、適当な数cipherに変換させる
    cipher = power(m,e,n)
    #数cipherをreturnする
    return cipher

def decrypt(pk, c):
     #2次元ベクトルpkから秘密鍵d、nの値を得る
    d, n = pk
    #テキストc本文を式 c^d mod nによって、適当な数plainに変換させる
    plain = power(c,d,n)
     #数plainをreturnする
    return plain
    
def RSAwithoutEDinput(p, q, message):
    print  ("元々のメッセージは:")
    print  (message)
    print  ("公開、秘密鍵は生成中. . .")
    public, private = generate_keypair(p, q)
    e, n =  public
    d, n =  private
    print  ("\n公開鍵は ", e ," \n秘密鍵は ", d)
    encrypted_msg = encrypt(public, message)
    print  ("\n暗号化した後のメッセージ: ")
    print  ( encrypted_msg)
    print  ("\n秘密鍵により複合する. . .")
    print  ("\n元々のメッセージは:")
    print  (decrypt(private, encrypted_msg))

def RSAwithEDinput(p, q, message, e, d):
    print  ("元々のメッセージは:")
    print  (message)
    n = p * q
    public=(e,n)
    private=(d,n)
    print  ("\n公開鍵は ", e ," \n秘密鍵は ", d)
    encrypted_msg = encrypt(public, message)
    print  ("\n暗号化した後のメッセージ: ")
    print  ( encrypted_msg)
    print  ("\n秘密鍵により複合する. . .")
    print  ("\n元々のメッセージは:")
    print  (decrypt(private, encrypted_msg))
    
def RSAwithoutEDinputTIME(p, q, message, kaisuu):
    timelist=[]
    public, private = generate_keypair(p, q)
    e, n =  public
    d, n =  private
    for i in range(1,kaisuu):
        t = time.perf_counter()
        encrypted_msg = encrypt(public, message)
        decrypt(private, encrypted_msg)
        timelist.append(time.perf_counter() - t)
    print('必要な時間の平均は :',np.average(timelist),'秒')
    print('必要な時間の分散は :',np.var(timelist),'秒^2')

def RSAwithEDinputTIME(p, q, message, e, d, kaisuu):
    timelist=[]
    n = p * q
    public=(e,n)
    private=(d,n)
    for i in range(1,kaisuu):
        t = time.perf_counter()
        encrypted_msg = encrypt(public, message)
        decrypt(private, encrypted_msg)
        timelist.append(time.perf_counter() - t)
    print('必要な時間の平均は :',np.average(timelist),'秒')
    print('必要な時間の分散は :',np.var(timelist),'秒^2')


In [2]:
#2.2-1
RSAwithEDinput(D9.p1 , D9.q1 , str(D2.m1), D9.e11 , D9.d11)   #e1-1はe11と名付けた、d1-1はd11と名付けた

元々のメッセージは:
123750811111151459123451714499640333335555544444

公開鍵は  576545171  
秘密鍵は  32401192536282284985293764068015617541088107068689912971046291803608281162104458551688806288958064348844691177528335638653613116641193673091528754858391486447179012634626996326273041718091639197392417588612717700648501308653623794225874395803030857098539375261982836116370926858180336861819322091831749480363

暗号化した後のメッセージ: 
23522580918314615869073196381805439371819359562176858399710182905141322427137243104966488088296483882531939480854785789851839785960148157008992035669755240571774311618123827617286042907477945574955286316184688060484215608093876256925272996463533961083816032728518978553697955256100012060716818878905008144912

公開鍵により複合する. . .

元々のメッセージは:
123750811111151459123451714499640333335555544444


In [3]:
#2.2-1
RSAwithEDinput(D10.p2 , D10.q2 , str(D2.m1), D10.e1 , D10.d1)

元々のメッセージは:
123750811111151459123451714499640333335555544444

公開鍵は  16906953396398285955  
秘密鍵は  724312013274593559104718233668131011194996096397911006981852189172961239048948863282836141926358773822231408712979119342947633487509489681999208855556699754187911367602954918028434935166295469845715934583725830138105140714265139027719788715269467483599026617823666098202900871395864967128743435198316095145

暗号化した後のメッセージ: 
72605607119066443461171293379214489489813185636091937276589617038651168819985260568048742729365928808467094179824789002010291422811169957006661108402143108244516407593337545213665877042899125416460685596284506651440560232839308883204338762297472990390199932833228556498918541599487837650255815478296123892906

公開鍵により複合する. . .

元々のメッセージは:
123750811111151459123451714499640333335555544444


In [4]:
#2.2-1
RSAwithEDinput(D11.p3 , D11.q3 , str(D2.m1), D11.e3 , D11.d3)

元々のメッセージは:
123750811111151459123451714499640333335555544444

公開鍵は  39420351215473146966470299931  
秘密鍵は  13937885823096711794538943920839289851419878930943691936364536860765115415925578795421424624560488515326679482876929479833861700946333089661036530657570281776821642299039218597495685008788366679109934826792030605510496883010641915892196047689485550568282679006488855011868603606388727010900645318095569480383

暗号化した後のメッセージ: 
48569633444799863279041018041448033271498997262597124510358556971011578407480012632953694397008242734206810148971002674500581896628686616597203610199346875852339422909001263504157907770237314436478692446953901361665354593806341549410575285379746360914055186863726873977154167274586683225523526111731242634527

公開鍵により複合する. . .

元々のメッセージは:
123750811111151459123451714499640333335555544444


In [5]:
#2.2-2
RSAwithEDinput(D9.p1 , D9.q1 ,str( D9.a1), D9.e11 , D9.d11)   #e1-1はe11と名付けた、d1-1はd11と名付けた

元々のメッセージは:
55171369910051437990412690780976918864004594400468057936995626399169935459645309293638273839011433220903844032978223080651001963165983148138903866465070349687717707483416960247609257698976748740151772071505763335844637258758436116165847460549578238068993246164273928592104593882489289102848385889827348866771

公開鍵は  576545171  
秘密鍵は  32401192536282284985293764068015617541088107068689912971046291803608281162104458551688806288958064348844691177528335638653613116641193673091528754858391486447179012634626996326273041718091639197392417588612717700648501308653623794225874395803030857098539375261982836116370926858180336861819322091831749480363

暗号化した後のメッセージ: 
87561627031437381414753926889955974181155248887405799607946794673084009285972201931506424319665713909839558177522875341212534437494321113569465668198375705787000488251215466180847686017492773339247678963462053583996419262103863350901616807236329105465109384394178147988431071613880231695911078763193518805353

公開鍵により複合する. . .

元々の

In [6]:
#2.2-2
RSAwithEDinput(D10.p2 , D10.q2 ,str( D10.a2), D10.e1 , D10.d1)

元々のメッセージは:
82731728725194908289223617666367457469528377629992344486892227999473808545734393601481638213883755082805006377065439788661243573340349538848236261335923042025459102737381949187014999261611684851439601167620664754455469880526500359543926950153918830779325957499251936408013440092257831969264506172196199532825

公開鍵は  16906953396398285955  
秘密鍵は  724312013274593559104718233668131011194996096397911006981852189172961239048948863282836141926358773822231408712979119342947633487509489681999208855556699754187911367602954918028434935166295469845715934583725830138105140714265139027719788715269467483599026617823666098202900871395864967128743435198316095145

暗号化した後のメッセージ: 
96154484235028072674542542161375237889803648278326976004169993188431727537796154591904488822942573935834804596016347871685521308678905515286027455197565544725004722369157901472504691905150834708767867909516053250903202069898309955837365025364299744363156868272739305009781974676019887182449964904282046823380

公開鍵により複合する.

In [7]:
#2.2-2
RSAwithEDinput(D11.p3 , D11.q3 , str(D11.a3),D11.e3 , D11.d3)

元々のメッセージは:
87671424421652388950642689643896199006445185897064638074058896168088599903264970862279686844170611506580163120454794848393134265598192406101533528611357631402234248276942302630353561038435876081389801027719173622195125722971702016017585000053477422925129871008047640445660779625633162226643547573746743342095

公開鍵は  39420351215473146966470299931  
秘密鍵は  13937885823096711794538943920839289851419878930943691936364536860765115415925578795421424624560488515326679482876929479833861700946333089661036530657570281776821642299039218597495685008788366679109934826792030605510496883010641915892196047689485550568282679006488855011868603606388727010900645318095569480383

暗号化した後のメッセージ: 
16911602025137316416710906982735468086448826814973389943996251590231573239251498378037919833321078330108014751962856643353972815686393990154048423259491497188779190499699715922080023854493664803348719047891835368599845389776664522273962125027472381743027511508617490646632370897831420874754198241995603035397



In [8]:
#2.2-3　　秒単位
RSAwithEDinputTIME(D9.p1 , D9.q1 , str(D2.m1), D9.e11 , D9.d11 ,1000)   #e1-1はe11と名付けた、d1-1はd11と名付けた

必要な時間の平均は : 0.30667863714414373 秒
必要な時間の分散は : 0.005932336673455994 秒^2


In [9]:
#2.2-3　　秒単位
RSAwithEDinputTIME(D10.p2 , D10.q2 , str(D2.m1), D10.e1 , D10.d1,1000)

必要な時間の平均は : 0.20950714562362308 秒
必要な時間の分散は : 0.005097699246095381 秒^2


In [10]:
#2.2-3　　秒単位
RSAwithEDinputTIME(D11.p3 , D11.q3 , str(D2.m1),D11.e3 , D11.d3,1000)

必要な時間の平均は : 0.19104567269970174 秒
必要な時間の分散は : 0.00042557449383076265 秒^2


In [11]:
#2.2-3　　秒単位
RSAwithEDinputTIME(D9.p1 , D9.q1 ,str( D9.a1), D9.e11 , D9.d11,1000)   #e1-1はe11と名付けた、d1-1はd11と名付けた

必要な時間の平均は : 0.18160547174875183 秒
必要な時間の分散は : 0.0011503697113634428 秒^2


In [12]:
#2.2-3　　秒単位
RSAwithEDinputTIME(D10.p2 , D10.q2 ,str( D10.a2), D10.e1 , D10.d1,1000)

必要な時間の平均は : 0.19952432617517596 秒
必要な時間の分散は : 0.0008589078769415838 秒^2


In [13]:
#2.2-3　　秒単位
RSAwithEDinputTIME(D11.p3 , D11.q3 , str(D11.a3),D11.e3 , D11.d3,1000)

必要な時間の平均は : 0.21327031605806115 秒
必要な時間の分散は : 0.0014112118347828367 秒^2
